In [ ]:
import pandas as pd
import os

In [ ]:
#path from which the data can be loaded into mysql database
MYSQL_PATH = r'C:\ProgramData\MySQL\MySQL Server 8.0\Data\kamernet_properties'

cwd = os.getcwd()

ROOT = os.path.realpath(os.path.join(cwd, '..'))
CSV_FILES = os.path.join(os.path.join(ROOT, "csv-files"))
DATA_PATH = 'original-dataset/properties.json'

### Load dataset into a dataframe

In [ ]:
df = pd.read_json(os.path.join(ROOT, DATA_PATH), lines= True)

In [ ]:
df["isRoomActive"].unique()

### Get data for cities model

In [ ]:
df_cities = df["city"]
df_cities.drop_duplicates(inplace=True)
df_cities = df_cities.sort_values()
df_cities = df_cities.reset_index(drop=True)
df_cities

Seems like there is "amsterdam" with lowercase letter at the beginning -> fix needed, but there are other priorities at the moment

In [ ]:
with open('..\DBschema&data\cities-insert.sql', 'w') as f:
    f.write("INSERT INTO kamernet_properties.cities (name)\nVALUES\n")
    for index, element in df_cities.iteritems():
        f.write(f"""("{element}")""")
        if index != df_cities.size-1:
            f.write(",\n")
        else:
            f.write(";")

In [ ]:
# filename = 'cities.csv'
# # default separator is ","
# # save to mysql server, without heading
# df_cities.to_csv(os.path.join(MYSQL_PATH, filename), header=False)
# # save to csv-files folder for reference, without heading
# df_cities.to_csv(os.path.join(CSV_FILES, filename), header=False)

### Get data for the Cost

In [ ]:
df.columns

In [ ]:
df_cost = df[["externalId","rent", "additionalCosts", "deposit"]]
#if there there is something missing replace with 0, this is most often the case with additional costs
df_cost = df_cost.fillna(0)
df_cost

In [ ]:
with open('..\DBschema&data\costs-insert.sql', 'w') as f:
    f.write("INSERT INTO kamernet_properties.costs (propertyID, rent, additionalCosts, deposit)\nVALUES\n")
    for idx, row in df_cost.iterrows():
        f.write(f"""("{row[0]}", {row[1]}, {row[2]}, {row[3]})""")
        if idx != df_cost.shape[0]-1:
            f.write(",\n")
        else:
            f.write(";")

### Get data for the Location

In [ ]:
df_location = df[["externalId","longitude", "latitude"]]
df_c = df["city"]
mapping = {value: index+1 for index, value in df_cities.iteritems()}
df_c = df_c.replace(mapping)
df_location = df_location.assign(cityID=df_c)
df_location["longitude"].unique()

In [ ]:
with open('..\DBschema&data\locations-insert.sql', 'w') as f:
    f.write("INSERT INTO kamernet_properties.locations (propertyID, longitude, latitude, cityID)\nVALUES\n")
    for idx, row in df_location.iterrows():
        f.write(f"""("{row[0]}", {row[1]}, {row[2]}, {row[3]})""")
        if idx != df_location.shape[0]-1:
            f.write(",\n")
        else:
            f.write(";")

### Get data for the Property

In [ ]:
df_property = df[["externalId", "isRoomActive", "areaSqm", "gender", "furnish", "coverImageUrl"]]
# leave them as strings
# mapping = {"Unfurnished": "false", "Furnished": "true"}
# df_property = df_property.replace({"furnish": mapping})
df_property["furnish"] = df_property["furnish"].fillna("Unknown")
# if isRoomActive is missing then the default value will be false
df_property["isRoomActive"] = df_property["isRoomActive"].fillna("false")
df_property = df_property.replace('', "Unknown")
df_property = df_property.merge(df_cost)
df_property = df_property.merge(df_location)
df_property["rentPerAreaSqm"] = df_property["rent"] / df_property["areaSqm"]
print(df_property["isRoomActive"].unique())

In [ ]:
with open('..\sql\properties-insert.sql', 'w') as f:
    f.write("INSERT INTO kamernet_properties.properties (propertyID, isRoomActive, areaSqm, gender, furnished, coverImageUrl, Cost_Rent, Cost_AdditionalCost, Cost_Deposit, Cost_RentPerAreaSqm, Location_Longitude, Location_Latitude, cityID)\nVALUES\n")
    for idx, row in df_property.iterrows():
        f.write(f"""("{row[0]}", {row[1]}, {row[2]}, "{row[3]}", "{row[4]}", "{row[5]}", {row[6]}, {row[7]}, {row[8]}, {row[12]}, {row[9]}, {row[10]}, {row[11]})""")
        if idx != df_property.shape[0]-1:
            f.write(",\n")
        else:
            f.write(";")